# Quality Model App

## Requirements

In [ ]:
%pip install ipywidgets==7.7.3

## App module and function

In [ ]:
from quality_model_widgets import QualityModelWidgets, create_input_data_dict

## Preparing Data

### Google Drive Mounting
Mount google drive to be used for storing the data.

In [ ]:
from google.colab import drive
drive.mount._DEBUG = False
drive.mount('/content/drive', force_remount=True)

### Specify the Data Location
Specify the URL to fetch from Mega

In [ ]:
data_url = https://mega.nz/file/sBwG1DqZ#NcN7Q97CJPh5DB-tXGb3a4SV6Bubw9xPFRfqPQ4bmw8
transfer_to_path = ''

### Transfer Data from Mega to Google Drive

In [ ]:
import os
import subprocess
import contextlib
from functools import wraps
import errno
import signal
import subprocess
import glob

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def latest_file(folder):
  list_of_files = glob.glob(f'{folder}/*') # * means all 
  latest_file = max(list_of_files, key=os.path.getctime)
  return latest_file

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfer(url):
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", url, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)
        
if not transfer_to_path:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"


class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator


@timeout(10)
def runShT(args):
    return runSh(args, output=True)

# def login(): 
#     runShT(f"mega-login {MEGA_USERNAME} {MEGA_PASSWORD}")

# #if the username and password provided then login to MEGA. 
# if MEGA_USERNAME != "" and MEGA_PASSWORD != "":
#     try:
#         login()
#     except TimeoutError:
#         runSh('mega-whoami', output=True)
# else:
#     print("Please Input your Mega IDs.")

transfer(data_url)
tagged_dataset_path = latest_file('./downloads')

## Specify Data Source and Output File

In [ ]:
input_dir = '../dataset/Tile_Generator_Genetic_Algo_V1_16x16-2023-23-2--16-01-20'
output_path = 'output.json'

## Run on Data Source

In [ ]:
data_dict = create_input_data_dict(input_dir)

## Widgets (Start the App)

In [ ]:
qualityModelWidgets = QualityModelWidgets(data_dict, output_path)
qualityModelWidgets.start()